In [1]:
import numpy as np
from Pyfhel import Pyfhel

In [2]:
# Example dataset
hours_studied = np.array([1.0, 2.0, 3.0, 4.0, 5.0], dtype=np.float64)  # Feature: Hours studied
exam_score = np.array([3.0, 5.0, 7.0, 9.0, 11.0], dtype=np.float64)    # Target: Exam score


In [3]:
# Feel free to change this number!
n_mults = 1

HE = Pyfhel(key_gen=True, context_params={
    'scheme': 'CKKS',
    'n': 2**14,         # For CKKS, n/2 values can be encoded in a single ciphertext. 
    'scale': 2**30,     # Each multiplication grows the final scale
    'qi_sizes': [60] + [30]*n_mults + [60] # Number of bits of each prime in the chain. 
                        # Intermediate prime sizes should be close to log2(scale).
                        # One per multiplication! More/higher qi_sizes means bigger 
                        #  ciphertexts and slower ops.
})

HE.relinKeyGen()
print("\nB1. CKKS context generation")
print(f"\t{HE}")


B1. CKKS context generation
	<ckks Pyfhel obj at 0x25c162b7a00, [pk:Y, sk:Y, rtk:-, rlk:Y, contx(n=16384, t=0, sec=128, qi=[60, 30, 60], scale=1073741824.0, )]>


In [4]:
#encrypting the hours, and the exam score
hours_studied_enc = HE.encryptFrac(hours_studied)
exam_score_enc = HE.encryptFrac(exam_score)


In [5]:
# implement a fucntion for secure multiplication
def secure_mul(HE, a, b, step, _r):
    for i in range(1,step+1):
        a *= b
        a = ~(a)
        print(f"Encrypted prediction (decrypted for demo): {_r(HE.decryptFrac(a))}")
        return a


In [6]:
# Number of observations
N = len(hours_studied)

# encrypt the number of observations
N_enc = HE.encryptFrac(np.array([N], dtype=np.float64))

# Function to round and truncate results for display
_r = lambda x: np.round(x, decimals=2)[:1]

# create a list for numerator_m ,denominator_m and c of np.float64 type
numerator_m = np.array(dtype=np.float64)
denominator_m = np.array( dtype=np.float64)
c = np.array(dtype=np.float64)

# Calculating the slope (m) and intercept (c) using the formula
numerator_m = HE.multiply(N_enc , ((HE.cumul_add(HE.multiply(hours_studied_enc , exam_score_enc ,in_new_ctxt=True), n_elements = 0 , in_new_ctxt=True)) - (HE.multiply(HE.cumul_add(hours_studied_enc, n_elements = 0 , in_new_ctxt=True) , HE.cumul_add(exam_score_enc, n_elements = 0 , in_new_ctxt=True), in_new_ctxt=True))), in_new_ctxt=True)
denominator_m = (N * np.sum(hours_studied**2)) - (np.sum(hours_studied)**2)
m = numerator_m / denominator_m

#intercept 
c = (np.sum(exam_score) - m * np.sum(hours_studied)) / N

print("slope = ", m, " intercept =", c)

# Correctly creating arrays filled with m and c
arr_m = np.full(shape=N, fill_value=m, dtype=np.float64) 
arr_c = np.full(shape=N, fill_value=c, dtype=np.float64)  # Corrected line


TypeError: array() missing required argument 'object' (pos 0)

Encrypting the slope and intercept with the same key HE

In [158]:
ctxt_w_enc  = HE.encryptFrac(arr_m)
ctxt_b_enc  = HE.encryptFrac(arr_c)

In [159]:
print("\nB2. Fixed-point Encoding & Encryption, ")
print("->\tarr_x ", hours_studied,'\n\t==> ctxt_x ', hours_studied_enc)


B2. Fixed-point Encoding & Encryption, 
->	arr_x  [1. 2. 3. 4. 5.] 
	==> ctxt_x  <Pyfhel Ciphertext at 0x1566e5b3e70, scheme=ckks, size=2/2, scale_bits=30, mod_level=0>


In [160]:
# Function to round and truncate results for display
_r = lambda x: np.round(x, decimals=2)[:N]

print("Performing encrypted predictions with relinearization for multiple data points:")


# Iterate over each encrypted hour studied
# 
n_nults = 1
for step in range(1,n_mults+1):
    hours_studied_enc *=  ctxt_w_enc  # Encrypted multiplication
    hours_studied_enc += ctxt_b_enc # encreypted addition
    hours_studied_enc = ~(hours_studied_enc)  # Relinearize after multiplication
    print(f"Encrypted prediction (decrypted for demo): {_r(HE.decryptFrac(hours_studied_enc))}")

    


Performing encrypted predictions with relinearization for multiple data points:
Encrypted prediction (decrypted for demo): [ 3.  5.  7.  9. 11.]


: 

In [130]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np

# Example dataset with a non-zero intercept
hours_studied = np.array([1, 2, 3, 4, 5])  # Independent variable
exam_score = np.array([3, 5, 7, 9, 11])    # Dependent variable (modified to change the intercept)

# Number of observations
N = len(hours_studied)

# Calculating the slope (m) and intercept (c) using the formula
numerator_m = (N * np.sum(hours_studied * exam_score)) - (np.sum(hours_studied) * np.sum(exam_score))
denominator_m = (N * np.sum(hours_studied**2)) - (np.sum(hours_studied)**2)
m = numerator_m / denominator_m

c = (np.sum(exam_score) - m * np.sum(hours_studied)) / N

print(f"Slope (m): {m}")
print(f"Intercept (c): {c}")

# Predicting exam scores
predicted_scores = m * hours_studied + c
print("Predicted Exam Scores:", predicted_scores)

# Optional: Calculating R-squared for model evaluation
ss_total = np.sum((exam_score - np.mean(exam_score))**2)
ss_res = np.sum((exam_score - predicted_scores)**2)
r_squared = 1 - (ss_res / ss_total)
print(f"R-squared: {r_squared}")


Slope (m): 2.0
Intercept (c): 1.0
Predicted Exam Scores: [ 3.  5.  7.  9. 11.]
R-squared: 1.0
